In [ ]:
# 对于数据处理的ResNet      
import tensorflow as tf                
from d2l import tensorflow as d2l           
class Residual(tf.keras.Model):  #@save                             
    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()                                                 
        self.conv1 = tf.keras.layers.Conv2D(
            num_channels, padding='same', kernel_size=3, strides=strides)
        self.conv2 = tf.keras.layers.Conv2D(
            num_channels, kernel_size=3, padding='same')
        self.conv3 = None
        if use_1x1conv:
            self.conv3 = tf.keras.layers.Conv2D(    
                num_channels, kernel_size=1, strides=strides)
        self.bn1 = tf.keras.layers.BatchNormalization() 
        self.bn2 = tf.keras.layers.BatchNormalization() 

    def call(self, X):
        Y = tf.keras.activations.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3 is not None:
            X = self.conv3(X)
        Y += X
        return tf.keras.activations.relu(Y)

class ResnetBlock(tf.keras.layers.Layer):
    def __init__(self, num_channels, num_residuals, first_block=False,
                 **kwargs):
        super(ResnetBlock, self).__init__(**kwargs)
        self.residual_layers = []          
        for i in range(num_residuals):
            if i == 0 and not first_block:
                self.residual_layers.append(
                    Residual(num_channels, use_1x1conv=True, strides=2))
            else:
                self.residual_layers.append(Residual(num_channels))

    def call(self, X):
        # for layer in self.residual_layers.layers:
         for layer in self.residual_layers:   
             X =layer(X) 
         return X 


# 调整网络结构
def net(lat_points, lon_ponints,channels=1, batch_size=60 ):    # 通道数为1 , 也就是只有降水这个变量. 
    # 定义网络结构 ==>   input_shape(batch_size , )      
    return tf.keras.Sequential([                                           # batch_size 就是 all_time_points = 45
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', input_shape=(lat_points, lon_ponints ,channels)),
        #  strides  要小于  kernel_size    
        # 可以这样理解,  channels 在实际物理中表示变量的个数.   
        tf.keras.layers.BatchNormalization(),     
        tf.keras.layers.Activation('relu'),             
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same'),   # ?L????????????????????
        ResnetBlock(64, 2, first_block=True),
        ResnetBlock(128, 2),
        ResnetBlock(256, 2),
        ResnetBlock(512, 2),
        tf.keras.layers.GlobalAvgPool2D(),
        tf.keras.layers.Dense(lat_points*lon_ponints, activation='relu')   # softmax   , 不必分类.   
    ])

b1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')])

b2 = ResnetBlock(64, 2, first_block=True)
b3 = ResnetBlock(128, 2)
b4 = ResnetBlock(256, 2)
b5 = ResnetBlock(512, 2)


# 训练模型 
# 自定义训练过程  ==> 暂时不用自定义训练过程, 直接使用fit方法
# @tf.function
# def train_step(inputs, labels, model, loss_fn, optimizer):
#     with tf.GradientTape() as tape:
#         predictions = model(inputs, training=True)
#         loss = loss_fn(labels, predictions)
#     gradients = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, model.trainable_variables))
#     return loss


# 编译模型
# 损失函数和优化器
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()   # 用于分类
loss_fn = tf.keras.losses.MeanSquaredError()   # 用于回归
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)        # 使用Adam优化器
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['mean_squared_error']) 



In [ ]:
# 创建模型实例
lat_pts, lon_pts, channels =  178, 156, 1  # 定义输入数据维度和通道数
model = net(lat_pts, lon_pts, channels)

In [ ]:
# 看一下大小维度的变化  : 
X = tf.random.uniform(shape=(60, 178,156, 1))  
for layer in net(178,156).layers:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

In [ ]:
# 加载数据集  
X_test = test_data.PRECT.values   + np.random.normal(0,1,60*178*156).reshape(60,178,156)                             
Y_test = test_data.PRECT.values   
X_train=train_data.PRECT.values + np.random.normal(0,1,60*178*156).reshape(60,178,156)                     
Y_train = train_data.PRECT.values

X_train = X_train[..., np.newaxis]   # 进行升维     ,   因为 ResNet 网络的输入要求是 4D 张量，channel 我设置的事1 , 所以需要增加一个维度
X_test = X_test[..., np.newaxis]
Y_train = np.reshape(Y_train, (60,-1))
Y_test = np.reshape(Y_test, (60,-1))
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


In [ ]:
# 训练模型
batch_size = 32     # 也就是多少个时刻.   样本数    
num_epochs = 10
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, validation_data=(X_test, Y_test)) 

# 绘制训练和验证的损失值
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])        # 验证集的损失值
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# 绘制训练和验证的准确率值
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])   # 验证集的准确率值          
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()  


# 绘制训练和验证的真实值和预测值的差异
plt.plot(Y_test, label='real')
plt.plot(model.predict(X_test), label='predict')
plt.title('Model predict')          
plt.legend()
plt.show()              

# 保存模型      
model.save('ResNet.h5')
